### This notebook shows examples of how to use the cyclops.processor API on GEMINI.

## Get all patient encounters from St. Michael's hospital from March 1 2020 to March 15 2020, with vitals and labs.

In [1]:
from cyclops.processors.constants import SMH
from cyclops.query import gemini

QUERY_SAVE_PATH = "/mnt/nfs/project/delirium/_extract/first_models/extract.h5"

# patients = gemini.patients(
#     hospitals=[SMH], from_date="2020-03-01", to_date="2020-03-15"
# )
# patients_vitals = gemini.events(patients=patients, category="vitals")
# patients_labs = gemini.events(patients=patients, category="labs")
# patients_vitals.run()
# patients_labs.run()

# print(f"{len(patients_vitals.data)} rows extracted!")
# print(f"{len(patients_labs.data)} rows extracted!")

# patients_vitals.data.to_hdf(
#     QUERY_SAVE_PATH,
#     key=f"query_demo_vitals",
# )
# patients_labs.data.to_hdf(
#     QUERY_SAVE_PATH,
#     key=f"query_demo_labs",
# )

2022-04-07 10:24:12,648 INFO cyclops.config  - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-07 10:24:13,092 INFO cyclops.query.utils - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-07 10:24:13,099 INFO cyclops.utils.profile - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-07 10:24:13,103 INFO cyclops.orm     - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-07 10:24:13,533 INFO cyclops.orm     - Database setup, ready to run queries!


## Process queried data into features. (Static aggregation + no imputation).

In [2]:
import pandas as pd

from cyclops.processor import Aggregator, Imputer
from cyclops.processor import featurize

vitals_data = pd.read_hdf(QUERY_SAVE_PATH, key=f"query_demo_vitals")
labs_data = pd.read_hdf(QUERY_SAVE_PATH, key=f"query_demo_labs")

feature_handler = featurize(data=[labs_data, vitals_data],
                            imputer=Imputer(strategy="none"),
                            aggregator=Aggregator(strategy="static",
                                                  range_=24,
                                                  window=24))

2022-04-07 10:24:13,569 INFO cyclops.processors.base - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-07 10:24:13,574 INFO cyclops.processors.admin - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-07 10:24:13,580 INFO cyclops.processors.labs - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-07 10:24:13,585 INFO cyclops.processors.vitals - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-07 10:24:14,152 INFO cyclops.processors.feature_handler - Log file is /mnt/nfs/home/krishnanam/vector-delirium/log.log
2022-04-07 10:24:14,534 INFO cyclops.processors.base - Processing raw admin data...
2022-04-07 10:24:14,538 INFO cyclops.processors.base - # samples: 16243, # encounters: 156
2022-04-07 10:24:14,540 INFO cyclops.processors.admin - # admin features: 2, # encounters: 156
2022-04-07 10:24:14,627 INFO cyclops.utils.profile - Finished executing function process in 0.092891 s
2022-04-07 10:24:14,636 INFO cycl

## Inspect feature handler, and check features.

In [3]:
print(feature_handler.get_numerical_features())
print(feature_handler.get_categorical_features())

feature_handler.features

['age', 'potassium', 'pt', 'sodium', 'urine sodium', 'urine specific gravity', 'bicarbonate', 'blood urea nitrogen', 'vitamin b12', 'white blood cell count', 'high sensitivity troponin', 'vitamin d', 'hba1c', 'lymphocyte', 'neutrophils', 'albumin', 'aptt', 'bilirubin', 'calcium', 'calcium, ionized', 'creatinine', 'esr', 'serum alcohol', 'ferritin', 'fibrinogen', 'glucose fasting', 'glucose point of care', 'glucose random', 'hematocrit', 'hemoglobin', 'crp', 'inr', 'ketone', 'lactate arterial', 'lactate venous', 'mean cell volume', 'arterial paco2', 'venous pco2', 'arterial ph', 'venous ph', 'platelet count', 'arterial pao2', 'CAM', 'Diastolic BP', 'Heart Rate', 'respiratory rate', 'Systolic BP', 'Temperature', 'weight']
['sex', 'fio2']


,age,sex,potassium,pt,sodium,urine sodium,urine specific gravity,bicarbonate,blood urea nitrogen,vitamin b12,...,platelet count,arterial pao2,CAM,Diastolic BP,Heart Rate,respiratory rate,Systolic BP,Temperature,weight,fio2
11102331,94,1,3.800000,NaN,142.000000,NaN,1.010,26.000000,NaN,507.0,...,158.000000,NaN,NaN,62.000000,88.000000,18.000000,111.000000,36.500000,NaN,1.0
11112372,43,0,4.000000,13.8,138.000000,NaN,NaN,26.000000,NaN,NaN,...,498.500000,NaN,0.0,78.500000,61.750000,18.000000,118.750000,35.533333,NaN,1.0
11105307,43,0,3.800000,18.6,133.500000,NaN,NaN,23.500000,NaN,NaN,...,78.000000,NaN,NaN,66.000000,91.000000,18.285714,111.714286,36.466667,NaN,1.0
11118152,48,1,4.500000,29.8,135.000000,NaN,1.035,27.000000,NaN,NaN,...,59.000000,NaN,NaN,76.000000,98.000000,20.000000,136.000000,35.500000,NaN,1.0
11156580,79,0,4.100000,NaN,137.000000,NaN,1.015,36.000000,NaN,NaN,...,375.000000,NaN,NaN,79.500000,86.500000,18.000000,134.500000,35.600000,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11996565,71,0,4.250000,NaN,132.000000,NaN,1.015,29.000000,NaN,357.0,...,204.000000,NaN,0.0,77.400000,108.500000,19.333333,132.000000,37.120000,1624.4,1.0
11968901,59,1,3.466667,13.2,116.166667,10.0,1.020,26.166667,NaN,NaN,...,197.333333,NaN,0.0,72.200000,95.000000,18.000000,106.200000,35.780000,62.2,1.0
11987993,49,0,3.866667,NaN,137.000000,NaN,NaN,26.333333,NaN,408.0,...,59.666667,NaN,0.0,88.750000,108.750000,19.333333,137.500000,36.533333,1094.0,1.0
11965564,70,0,4.175000,NaN,129.000000,44.0,1.014,25.800000,NaN,NaN,...,389.000000,NaN,0.0,101.666667,84.333333,22.000000,157.666667,36.466667,NaN,1.0


## Look at data quality report (patient statics + labs + vitals)

In [4]:
from evidently import ColumnMapping
from evidently.dashboard import Dashboard
from evidently.dashboard.tabs import DataQualityTab

dashboard = Dashboard(tabs=[DataQualityTab()])
column_mapping = ColumnMapping(
    numerical_features=feature_handler.get_numerical_features(),
    categorical_features=feature_handler.get_categorical_features(),
)
dashboard.calculate(
    reference_data=feature_handler.features,
    current_data=None,
    column_mapping=column_mapping,
)
dashboard.save(f"data_processed.html")